#### Lucas-Kanade 알고리즘
- 이웃 화소는 유사한 모션 벡터를 가져야 한다

In [1]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [8]:
cap = cv2.VideoCapture('/content/drive/MyDrive/2025_1_Colab/ComputerVision_Pattern/images/slow_traffic_small.mp4')

In [3]:
def draw_OpticalFlow(img, flow, step = 16):
  for y in range(step//2, img.shape[0], step):
    for x in range(step//2, img.shape[1], step):
      dx, dy = flow[y, x].astype(np.int32)
      if (dx*dx+dy*dy) > 1:
        cv2.line(img, (x, y), (x + dx, y + dy), (0, 0, 255), 2)
      else:
        cv2.line(img, (x, y), (x + dx, y + dy), (255, 0, 0), 2)

In [4]:
prev = None
while True:
  ret, frame = cap.read()
  if not ret:
    break
  if prev is None:
    prev = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    continue

  curr = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  flow = cv2.calcOpticalFlowFarneback(prev, curr, None, 0.5, 3, 15, 3, 5, 1.2, 0)

  draw_OpticalFlow(frame, flow)
  cv2_imshow(frame)

Output hidden; open in https://colab.research.google.com to view.

### KLT
- 광류 정보를 이용하여 지역 특징을 추적하는 방식
- 지역 특징에서만 모션 벡터를 추정하기 때문에 희소 광류라 부름
- 큰 이동을 처리하기 위해 피라미드 영상 활용
- OpenCV는 지역 특징을 추출하는 goodFeaturesToTrack 함수와 광류와 추적 정보를 계산하는 calcOpticalFlowPyrLK 함수 제공

In [6]:
import matplotlib.pyplot as plt

In [9]:
feature_params = dict(maxCorners = 100, qualityLevel = 0.3, minDistance = 7, blockSize = 7)
lk_params = dict(winSize = (15, 15), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

color = np.random.randint(0, 255, (100, 3))
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

mask = np.zeros_like(old_frame)

In [10]:
while(1):
  ret, frame = cap.read()
  if not ret:
    break

  new_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  p1, match, err = cv2.calcOpticalFlowPyrLK(old_gray, new_gray, p0, None, **lk_params)

  if p1 is not None:
    good_new = p1[match == 1]
    good_old = p0[match == 1]

  for i in range(len(good_new)):
    a, b = int(good_new[i][0]), int(good_new[i][1])
    c, d = int(good_old[i][0]), int(good_old[i][1])
    mask = cv2.line(mask, (a, b), (c, d), color[i].tolist(), 2)
    frame = cv2.circle(frame, (a, b), 5, color[i].tolist(), -1)

  img = cv2.add(frame, mask)
  cv2_imshow(img)

  old_gray = new_gray.copy()
  p0 = good_new.reshape(-1, 1, 2)

Output hidden; open in https://colab.research.google.com to view.